# 数据预处理

列名及其解释

| **dt** | **kdcode** | **volume** | **turnover** | **turn** | **pct_chg**    | **vwap**   | **trade_status**                                             | **adj_close**  | **adj_open**     | **adj_high**   | **adj_low**    |
| ------ | ---------- | ---------- | ------------ | -------- | -------------- | ---------- | ------------------------------------------------------------ | -------------- | ---------------- | -------------- | -------------- |
| 日期   | 股票代码   | 成交量     | 成交额       | 换手率   | 涨跌幅度百分数 | 平均成交价 | 交易状态:0 正常；100 停牌一天；1 盘中停牌；2 停牌1小时以内，3停牌半天；101 其他 | 复权后的收盘价 | 复权后的开盘价格 | 复权后的最高价 | 复权后的最低价 |


In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
df = pd.read_csv('data2.csv')
df = df.dropna(axis=0)
df

/home/tianhao/env4tiho_py36/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
Duplicate key in file PosixPath('/home/tianhao/env4tiho_py36/lib/python3.6/site-packages/matplotlib/mpl-data/matplotlibrc'), line 400 ('axes.unicode_minus: True  # use Unicode for the minus symbol rather than hyphen.  See')
/home/tianhao/env4tiho_py36/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,Unnamed: 0,dt,kdcode,volume,turnover,turn,pct_chg,vwap,trade_status,adj_close,adj_open,adj_high,adj_low,label
0,0,2016-01-04,000001.SZ,56349787.0,6.603762e+08,0.4774,-5.51,11.719231,0,935.197763,990.500720,992.976972,926.943591,0.0
1,1,2016-01-05,000001.SZ,66326995.0,7.555314e+08,0.5619,0.61,11.391008,0,940.975684,930.245260,955.007778,920.340253,0.0
2,2,2016-01-06,000001.SZ,51570644.0,5.916985e+08,0.4369,1.14,11.473553,0,951.706109,942.626519,954.182361,940.150267,0.0
3,3,2016-01-07,000001.SZ,17476110.0,1.948695e+08,0.1481,-5.12,11.150622,0,903.006490,941.801102,941.801102,900.530238,0.0
4,4,2016-01-08,000001.SZ,74752758.0,8.313345e+08,0.6333,1.64,11.121122,0,917.864001,925.292756,931.896094,899.704821,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4676431,4676431,2021-07-26,688981.SH,97665400.0,5.346218e+09,5.5900,10.25,54.740148,0,57.200000,51.850000,57.320000,51.120000,1.0
4676432,4676432,2021-07-27,688981.SH,308012900.0,1.993186e+10,16.4600,12.59,64.711114,0,64.400000,57.400000,68.640000,57.400000,1.0
4676433,4676433,2021-07-28,688981.SH,172850500.0,1.044512e+10,9.2400,-7.98,60.428625,0,59.260000,62.950000,63.490000,58.280000,1.0
4676434,4676434,2021-07-29,688981.SH,144274400.0,8.933641e+09,7.7100,5.08,61.921177,0,62.270000,60.400000,64.580000,59.370000,1.0


## 0. 添加开盘价涨跌标签`label`

label可以尝试使用：
* label = 后一天adj_open - 当天adj_open
* label = 三天后adj_open - 当天adj_open
* label = 三天后平均adj_open - 当天adj_open (TODO 暂未尝试)

---
**方式1** label = 后一天adj_open - 当天adj_open

In [15]:
df = clean_df
df_open2 = df['adj_open'].iloc[1:].values
df_open1 = df['adj_open'].iloc[:-1].values

In [16]:
pre_label = df_open2 - df_open1
pre_label

array([-60.25546048,  12.381259  ,  -0.82541727, ...,   5.55      ,
        -2.55      ,   0.85      ])

In [17]:
pre_label[pre_label >= 0] = 1
pre_label[pre_label < 0] = 0
pre_label

array([0., 1., 0., ..., 1., 0., 1.])

In [18]:
pre_label = list(pre_label)
pre_label.append(0)
df['label'] = pre_label

/home/tianhao/env4tiho_py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
df

,kdcode,volume,turnover,turn,pct_chg,vwap,trade_status,adj_close,adj_open,adj_high,adj_low,label
dt,,,,,,,,,,,,
2016-01-04,000001.SZ,56349787.0,6.603762e+08,0.4774,-5.51,11.719231,0,935.197763,990.500720,992.976972,926.943591,0.0
2016-01-05,000001.SZ,66326995.0,7.555314e+08,0.5619,0.61,11.391008,0,940.975684,930.245260,955.007778,920.340253,1.0
2016-01-06,000001.SZ,51570644.0,5.916985e+08,0.4369,1.14,11.473553,0,951.706109,942.626519,954.182361,940.150267,0.0
2016-01-07,000001.SZ,17476110.0,1.948695e+08,0.1481,-5.12,11.150622,0,903.006490,941.801102,941.801102,900.530238,0.0
2016-01-08,000001.SZ,74752758.0,8.313345e+08,0.6333,1.64,11.121122,0,917.864001,925.292756,931.896094,899.704821,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-26,688981.SH,97665400.0,5.346218e+09,5.5900,10.25,54.740148,0,57.200000,51.850000,57.320000,51.120000,1.0
2021-07-27,688981.SH,308012900.0,1.993186e+10,16.4600,12.59,64.711114,0,64.400000,57.400000,68.640000,57.400000,1.0
2021-07-28,688981.SH,172850500.0,1.044512e+10,9.2400,-7.98,60.428625,0,59.260000,62.950000,63.490000,58.280000,0.0


In [20]:
df.to_csv('data.csv')

**方式2** label = 三天后adj_open - 当天adj_open

In [21]:
import pandas as pd
df = pd.read_csv('data2.csv')
df

,dt,kdcode,volume,turnover,turn,pct_chg,vwap,trade_status,adj_close,adj_open,adj_high,adj_low,label
0,2016-01-04,000001.SZ,56349787.0,6.603762e+08,0.4774,-5.51,11.719231,0,935.197763,990.500720,992.976972,926.943591,0.0
1,2016-01-05,000001.SZ,66326995.0,7.555314e+08,0.5619,0.61,11.391008,0,940.975684,930.245260,955.007778,920.340253,1.0
2,2016-01-06,000001.SZ,51570644.0,5.916985e+08,0.4369,1.14,11.473553,0,951.706109,942.626519,954.182361,940.150267,0.0
3,2016-01-07,000001.SZ,17476110.0,1.948695e+08,0.1481,-5.12,11.150622,0,903.006490,941.801102,941.801102,900.530238,0.0
4,2016-01-08,000001.SZ,74752758.0,8.313345e+08,0.6333,1.64,11.121122,0,917.864001,925.292756,931.896094,899.704821,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4676431,2021-07-26,688981.SH,97665400.0,5.346218e+09,5.5900,10.25,54.740148,0,57.200000,51.850000,57.320000,51.120000,1.0
4676432,2021-07-27,688981.SH,308012900.0,1.993186e+10,16.4600,12.59,64.711114,0,64.400000,57.400000,68.640000,57.400000,1.0
4676433,2021-07-28,688981.SH,172850500.0,1.044512e+10,9.2400,-7.98,60.428625,0,59.260000,62.950000,63.490000,58.280000,0.0
4676434,2021-07-29,688981.SH,144274400.0,8.933641e+09,7.7100,5.08,61.921177,0,62.270000,60.400000,64.580000,59.370000,1.0


In [22]:
df_open2 = df['adj_open'].iloc[3:].values
df_open1 = df['adj_open'].iloc[:-3].values

In [23]:
pre_label = df_open2 - df_open1
pre_label

array([-48.69961875,  -4.9525036 , -34.66752521, ...,   9.95      ,
         8.55      ,   3.85      ])

In [24]:
pre_label[pre_label >= 0] = 1
pre_label[pre_label < 0] = 0
pre_label

array([0., 0., 0., ..., 1., 1., 1.])

In [25]:
pre_label = list(pre_label)
pre_label.append(1)
pre_label.append(1)
pre_label.append(1)
df['label'] = pre_label

In [26]:
df

,dt,kdcode,volume,turnover,turn,pct_chg,vwap,trade_status,adj_close,adj_open,adj_high,adj_low,label
0,2016-01-04,000001.SZ,56349787.0,6.603762e+08,0.4774,-5.51,11.719231,0,935.197763,990.500720,992.976972,926.943591,0.0
1,2016-01-05,000001.SZ,66326995.0,7.555314e+08,0.5619,0.61,11.391008,0,940.975684,930.245260,955.007778,920.340253,0.0
2,2016-01-06,000001.SZ,51570644.0,5.916985e+08,0.4369,1.14,11.473553,0,951.706109,942.626519,954.182361,940.150267,0.0
3,2016-01-07,000001.SZ,17476110.0,1.948695e+08,0.1481,-5.12,11.150622,0,903.006490,941.801102,941.801102,900.530238,0.0
4,2016-01-08,000001.SZ,74752758.0,8.313345e+08,0.6333,1.64,11.121122,0,917.864001,925.292756,931.896094,899.704821,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4676431,2021-07-26,688981.SH,97665400.0,5.346218e+09,5.5900,10.25,54.740148,0,57.200000,51.850000,57.320000,51.120000,1.0
4676432,2021-07-27,688981.SH,308012900.0,1.993186e+10,16.4600,12.59,64.711114,0,64.400000,57.400000,68.640000,57.400000,1.0
4676433,2021-07-28,688981.SH,172850500.0,1.044512e+10,9.2400,-7.98,60.428625,0,59.260000,62.950000,63.490000,58.280000,1.0
4676434,2021-07-29,688981.SH,144274400.0,8.933641e+09,7.7100,5.08,61.921177,0,62.270000,60.400000,64.580000,59.370000,1.0


In [27]:
df.to_csv('data2.csv')

## 1. 根据股票代码划分数据

In [2]:
stocks_code = df['kdcode'].unique()
stocks_num = len(stocks_code)
"股票数量",stocks_num

('股票数量', 4018)

In [3]:
# 根据股票代码划分数据
for i, stock_i in enumerate(tqdm(stocks_code)):
    stock_i_data = df[df['kdcode'].isin([stock_i])]
    exec("df%s = stock_i_data" % i)

100%|██████████| 4018/4018 [03:47<00:00, 17.65it/s]


In [4]:
res_df = pd.DataFrame()
res_df['volume'] = df['volume']
res_df['turnover'] = df['turnover']
res_df['turn'] = df['turn']
res_df

,volume,turnover,turn
0,56349787.0,6.603762e+08,0.4774
1,66326995.0,7.555314e+08,0.5619
2,51570644.0,5.916985e+08,0.4369
3,17476110.0,1.948695e+08,0.1481
4,74752758.0,8.313345e+08,0.6333
...,...,...,...
4676431,97665400.0,5.346218e+09,5.5900
4676432,308012900.0,1.993186e+10,16.4600
4676433,172850500.0,1.044512e+10,9.2400
4676434,144274400.0,8.933641e+09,7.7100


## 2. 按照股票添加特征

### 2.1 添加1日、3日、5日、10、20日`vwap`均价特征

In [5]:
res_df['vwap'] = df['vwap'] # 直接放入1日的
sizes = [3, 5, 10, 20]

In [6]:
for t, size in enumerate(sizes):
    exec("meanList%s = []" % t)
    for i in tqdm(range(stocks_num), desc='遍历所有股票'): # 循环股票
        exec("stock = df%s" % i)
        vwap = stock['vwap'].values # 成交价
        for index in range(stock.shape[0]):
            if index < size - 1:
                mean_res = vwap[:index + 1].mean()
            else:
                mean_res = vwap[index - size + 1:index + 1].mean()
            exec("meanList%s.append(%s)" % (t, mean_res))

res_df['vwap_mean3'] = meanList0
res_df['vwap_mean5'] = meanList1
res_df['vwap_mean10'] = meanList2
res_df['vwap_mean20'] = meanList3

遍历所有股票: 100%|██████████| 4018/4018 [01:26<00:00, 46.29it/s] 


### 2.2 添加1日、3日、5日、10日平均涨跌幅`pct_chg`

In [7]:
res_df['pct_chg'] = df['pct_chg']
sizes = [3, 5, 10]

for t, size in enumerate(sizes):
    exec("meanList%s = []" % t)
    for i in tqdm(range(stocks_num), desc='遍历所有股票'): # 循环股票
        exec("stock = df%s" % i)
        pct_chg = stock['pct_chg'].values # 涨跌幅
        for index in range(stock.shape[0]):
            if index < size - 1:
                mean_res = pct_chg[:index + 1].mean()
            else:
                mean_res = pct_chg[index - size + 1:index + 1].mean()
            exec("meanList%s.append(%s)" % (t, mean_res))

res_df['chg_mean3'] = meanList0
res_df['chg_mean5'] = meanList1
res_df['chg_mean10'] = meanList2

遍历所有股票: 100%|██████████| 4018/4018 [01:34<00:00, 42.55it/s] 


In [8]:
res_df

,volume,turnover,turn,vwap,vwap_mean3,vwap_mean5,vwap_mean10,pct_chg,chg_mean3,chg_mean5,chg_mean10
0,56349787.0,6.603762e+08,0.4774,11.719231,11.719231,11.719231,11.719231,-5.51,-5.510000,-5.510000,-5.510000
1,66326995.0,7.555314e+08,0.5619,11.391008,11.555119,11.555119,11.555119,0.61,-2.450000,-2.450000,-2.450000
2,51570644.0,5.916985e+08,0.4369,11.473553,11.527931,11.527931,11.527931,1.14,-1.253333,-1.253333,-1.253333
3,17476110.0,1.948695e+08,0.1481,11.150622,11.338394,11.433604,11.433604,-5.12,-1.123333,-2.220000,-2.220000
4,74752758.0,8.313345e+08,0.6333,11.121122,11.248432,11.371107,11.371107,1.64,-0.780000,-1.448000,-1.448000
...,...,...,...,...,...,...,...,...,...,...,...
4676431,97665400.0,5.346218e+09,5.5900,54.740148,53.319652,52.787750,53.961543,10.25,2.966667,2.088000,0.170000
4676432,308012900.0,1.993186e+10,16.4600,64.711114,57.224642,55.416810,54.720099,12.59,6.910000,4.576000,1.353000
4676433,172850500.0,1.044512e+10,9.2400,60.428625,59.959962,57.019739,55.101627,-7.98,4.953333,2.702000,0.737000
4676434,144274400.0,8.933641e+09,7.7100,61.921177,62.353639,58.804745,55.757486,5.08,3.230000,3.566000,1.393000


### 2.3 添加1日、3日、5日、10日平均开盘价

In [9]:
res_df['adj_open'] = df['adj_open']
sizes = [3, 5, 10]
for t, size in enumerate(sizes):
    exec("meanList%s = []" % t)
    for i in tqdm(range(stocks_num), desc='遍历所有股票'): # 循环股票
        exec("stock = df%s" % i)
        adj_open = stock['adj_open'].values # 开盘价
        for index in range(stock.shape[0]):
            if index < size - 1:
                mean_res = adj_open[:index + 1].mean()
            else:
                mean_res = adj_open[index - size + 1:index + 1].mean()
            exec("meanList%s.append(%s)" % (t, mean_res))

res_df['open_mean3'] = meanList0
res_df['open_mean5'] = meanList1
res_df['open_mean10'] = meanList2

遍历所有股票: 100%|██████████| 4018/4018 [01:26<00:00, 46.34it/s] 


In [10]:
res_df

,volume,turnover,turn,vwap,vwap_mean3,vwap_mean5,vwap_mean10,pct_chg,chg_mean3,chg_mean5,chg_mean10,adj_open,open_mean3,open_mean5,open_mean10
0,56349787.0,6.603762e+08,0.4774,11.719231,11.719231,11.719231,11.719231,-5.51,-5.510000,-5.510000,-5.510000,990.500720,990.500720,990.500720,990.500720
1,66326995.0,7.555314e+08,0.5619,11.391008,11.555119,11.555119,11.555119,0.61,-2.450000,-2.450000,-2.450000,930.245260,960.372990,960.372990,960.372990
2,51570644.0,5.916985e+08,0.4369,11.473553,11.527931,11.527931,11.527931,1.14,-1.253333,-1.253333,-1.253333,942.626519,954.457500,954.457500,954.457500
3,17476110.0,1.948695e+08,0.1481,11.150622,11.338394,11.433604,11.433604,-5.12,-1.123333,-2.220000,-2.220000,941.801102,938.224293,951.293400,951.293400
4,74752758.0,8.313345e+08,0.6333,11.121122,11.248432,11.371107,11.371107,1.64,-0.780000,-1.448000,-1.448000,925.292756,936.573459,946.093271,946.093271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4676431,97665400.0,5.346218e+09,5.5900,54.740148,53.319652,52.787750,53.961543,10.25,2.966667,2.088000,0.170000,51.850000,52.496667,52.078000,53.945000
4676432,308012900.0,1.993186e+10,16.4600,64.711114,57.224642,55.416810,54.720099,12.59,6.910000,4.576000,1.353000,57.400000,54.083333,53.358000,54.024000
4676433,172850500.0,1.044512e+10,9.2400,60.428625,59.959962,57.019739,55.101627,-7.98,4.953333,2.702000,0.737000,62.950000,57.400000,55.568000,54.614000
4676434,144274400.0,8.933641e+09,7.7100,61.921177,62.353639,58.804745,55.757486,5.08,3.230000,3.566000,1.393000,60.400000,60.250000,57.120000,55.052000


### 2.4 添加1、3、5、10日的顺势指标CCI

In [11]:
# sizes = [1, 3, 5, 10]

# def getCCI(data, ndays):
#     TP = (data['adj_high'] + data['adj_low'] + data['adj_close']) / 3 
#     CCI = pd.Series((TP - TP.rolling(ndays).mean()) / (0.015 * TP.rolling(ndays).mean()),name = 'CCI')
#     return CCI

# for t, size in enumerate(sizes):
#     CCISeries = pd.Series()
#     for i in tqdm(range(stocks_num), desc='遍历所有股票'): # 循环股票
#         exec("stock = df%s" % i)
#         CCI = getCCI(stock, size)
#         for index in range(size - 1):
#             cci = getCCI(stock.iloc[:index + 1], index + 1).iloc[index]
#             CCI.iloc[index] = cci
#         CCISeries = pd.concat([CCISeries, CCI])
#     print(CCISeries)
#     print("max: ", CCISeries.max(), "min: ", CCISeries.min())
#     res_df['CCI_' + str(size)] = CCISeries

# res_df

### 2.5 添加1、3、5、10日的简易波动指标EMV

In [12]:
# sizes = [1, 3, 5, 10]

# def getEVM(data, ndays):
#     dm = ((data['adj_high'] + data['adj_low'])/2) - ((data['adj_high'].shift(1) + data['adj_low'].shift(1))/2)
#     br = (data['volume']) / ((data['adj_high'] - data['adj_low']))
#     EVM = dm / br
#     EVM_MA = pd.Series(EVM.rolling(ndays).mean(), name = 'EVM')
#     return EVM_MA

# for t, size in enumerate(sizes):
#     EVMSeries = pd.Series()
#     for i in tqdm(range(stocks_num), desc='遍历所有股票'): # 循环股票
#         exec("stock = df%s" % i)
#         EVM = getEVM(stock, size)
#         for index in range(size - 1):
#             evm = getEVM(stock.iloc[:index + 1], index + 1).iloc[index]
#             EVM.iloc[index] = evm
#         EVMSeries = pd.concat([EVMSeries, EVM])
#     print(EVMSeries)
#     print("max: ", EVMSeries.max(), "min: ", EVMSeries.min())
#     res_df['EVM_' + str(size)] = EVMSeries
        
# res_df

### 2.6 添加1、3、5、10日的强力指数FI

In [13]:
# sizes = [1, 3, 5, 10]
# def getFI(data, ndays):
#     FI = pd.Series(data['adj_close'].diff(ndays) * data['volume'], name = 'ForceIndex') 
#     return FI

# for t, size in enumerate(sizes):
#     FISeries = pd.Series()
#     for i in tqdm(range(stocks_num), desc='遍历所有股票'): # 循环股票
#         exec("stock = df%s" % i)
#         FI = getFI(stock, size)
#         for index in range(size - 1):
#             fi = getFI(stock.iloc[:index + 1], index + 1).iloc[index]
#             FI.iloc[index] = fi
#         FISeries = pd.concat([FISeries, FI])
#     print(FISeries)
# #     FISeries = FISeries / 100000000
#     print("max: ", FISeries.max(), "min: ", FISeries.min())
#     res_df['FI_' + str(size)] = FISeries
# res_df

### 2.7 添加`label`

## 4. 归一化

In [18]:
tmp_df = res_df.copy()

In [19]:
from sklearn import preprocessing

# test(train)Data3/4
robust_scaler = preprocessing.RobustScaler()
_df = robust_scaler.fit_transform(res_df)

# # test(train)Data1/2
# min_max_scaler = preprocessing.MinMaxScaler()
# _df = min_max_scaler.fit_transform(res_df)

_df = pd.DataFrame(_df, columns=res_df.columns)
_df

,volume,turnover,turn,vwap,vwap_mean3,vwap_mean5,vwap_mean10,pct_chg,chg_mean3,chg_mean5,chg_mean10,adj_open,open_mean3,open_mean5,open_mean10
0,4.523495,4.509965,-0.370305,0.064544,0.064377,0.064083,0.063036,-2.155881,-3.487342,-4.358350,-6.033728,18.997222,18.994527,18.993194,18.989172
1,5.411601,5.229687,-0.333054,0.040246,0.052216,0.051918,0.050869,0.238673,-1.550633,-1.937923,-2.682873,17.793307,18.392643,18.391340,18.387424
2,4.098086,3.990511,-0.388159,0.046357,0.050202,0.049903,0.048853,0.446044,-0.793249,-0.991373,-1.372463,18.040687,18.274465,18.273168,18.269272
3,1.063213,0.989032,-0.515473,0.022451,0.036157,0.042911,0.041859,-2.003287,-0.710970,-1.755996,-2.431012,18.024195,17.950162,18.209960,18.206075
4,6.161607,5.803036,-0.301578,0.020267,0.029490,0.038279,0.037226,0.641678,-0.493671,-1.145352,-1.585633,17.694355,17.917182,18.106078,18.102212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4676431,8.201142,39.952081,1.883530,3.249302,3.147030,3.108206,3.194988,4.010486,1.877637,1.651585,0.186159,0.242811,0.255329,0.246581,0.283132
4676432,26.924907,150.272891,6.675454,3.987435,3.436396,3.303080,3.251229,4.926051,4.373418,3.619566,1.481603,0.353701,0.287027,0.272151,0.284710
4676433,14.893630,78.518421,3.492594,3.670410,3.639087,3.421894,3.279517,-3.122310,3.135021,2.137252,0.807052,0.464591,0.353287,0.316300,0.296494
4676434,12.349971,67.086121,2.818110,3.780901,3.816462,3.554204,3.328144,1.987636,2.044304,2.820667,1.525405,0.413642,0.410223,0.347303,0.305242


In [20]:
# 添加不需归一化的列
_df['label'] = df['label']
_df['kdcode'] = df['kdcode']
_df['dt'] = df['dt']

order = ['dt','kdcode','volume','turnover','turn','vwap','vwap_mean3','vwap_mean5','vwap_mean10','vwap_mean20','pct_chg','chg_mean3','chg_mean5','chg_mean10','adj_open','open_mean3','open_mean5','open_mean10','label']
df_ = _df[order]
df_

,dt,kdcode,volume,turnover,turn,vwap,vwap_mean3,vwap_mean5,vwap_mean10,pct_chg,chg_mean3,chg_mean5,chg_mean10,adj_open,open_mean3,open_mean5,open_mean10,label
0,2016-01-04,000001.SZ,4.523495,4.509965,-0.370305,0.064544,0.064377,0.064083,0.063036,-2.155881,-3.487342,-4.358350,-6.033728,18.997222,18.994527,18.993194,18.989172,0.0
1,2016-01-05,000001.SZ,5.411601,5.229687,-0.333054,0.040246,0.052216,0.051918,0.050869,0.238673,-1.550633,-1.937923,-2.682873,17.793307,18.392643,18.391340,18.387424,0.0
2,2016-01-06,000001.SZ,4.098086,3.990511,-0.388159,0.046357,0.050202,0.049903,0.048853,0.446044,-0.793249,-0.991373,-1.372463,18.040687,18.274465,18.273168,18.269272,0.0
3,2016-01-07,000001.SZ,1.063213,0.989032,-0.515473,0.022451,0.036157,0.042911,0.041859,-2.003287,-0.710970,-1.755996,-2.431012,18.024195,17.950162,18.209960,18.206075,0.0
4,2016-01-08,000001.SZ,6.161607,5.803036,-0.301578,0.020267,0.029490,0.038279,0.037226,0.641678,-0.493671,-1.145352,-1.585633,17.694355,17.917182,18.106078,18.102212,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4676431,2021-07-26,688981.SH,8.201142,39.952081,1.883530,3.249302,3.147030,3.108206,3.194988,4.010486,1.877637,1.651585,0.186159,0.242811,0.255329,0.246581,0.283132,1.0
4676432,2021-07-27,688981.SH,26.924907,150.272891,6.675454,3.987435,3.436396,3.303080,3.251229,4.926051,4.373418,3.619566,1.481603,0.353701,0.287027,0.272151,0.284710,1.0
4676433,2021-07-28,688981.SH,14.893630,78.518421,3.492594,3.670410,3.639087,3.421894,3.279517,-3.122310,3.135021,2.137252,0.807052,0.464591,0.353287,0.316300,0.296494,1.0
4676434,2021-07-29,688981.SH,12.349971,67.086121,2.818110,3.780901,3.816462,3.554204,3.328144,1.987636,2.044304,2.820667,1.525405,0.413642,0.410223,0.347303,0.305242,1.0


In [21]:
# df_ = df_.dropna(axis=0)
df_ = df_.fillna(0)
df_

,dt,kdcode,volume,turnover,turn,vwap,vwap_mean3,vwap_mean5,vwap_mean10,pct_chg,chg_mean3,chg_mean5,chg_mean10,adj_open,open_mean3,open_mean5,open_mean10,label
0,2016-01-04,000001.SZ,4.523495,4.509965,-0.370305,0.064544,0.064377,0.064083,0.063036,-2.155881,-3.487342,-4.358350,-6.033728,18.997222,18.994527,18.993194,18.989172,0.0
1,2016-01-05,000001.SZ,5.411601,5.229687,-0.333054,0.040246,0.052216,0.051918,0.050869,0.238673,-1.550633,-1.937923,-2.682873,17.793307,18.392643,18.391340,18.387424,0.0
2,2016-01-06,000001.SZ,4.098086,3.990511,-0.388159,0.046357,0.050202,0.049903,0.048853,0.446044,-0.793249,-0.991373,-1.372463,18.040687,18.274465,18.273168,18.269272,0.0
3,2016-01-07,000001.SZ,1.063213,0.989032,-0.515473,0.022451,0.036157,0.042911,0.041859,-2.003287,-0.710970,-1.755996,-2.431012,18.024195,17.950162,18.209960,18.206075,0.0
4,2016-01-08,000001.SZ,6.161607,5.803036,-0.301578,0.020267,0.029490,0.038279,0.037226,0.641678,-0.493671,-1.145352,-1.585633,17.694355,17.917182,18.106078,18.102212,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4676431,2021-07-26,688981.SH,8.201142,39.952081,1.883530,3.249302,3.147030,3.108206,3.194988,4.010486,1.877637,1.651585,0.186159,0.242811,0.255329,0.246581,0.283132,1.0
4676432,2021-07-27,688981.SH,26.924907,150.272891,6.675454,3.987435,3.436396,3.303080,3.251229,4.926051,4.373418,3.619566,1.481603,0.353701,0.287027,0.272151,0.284710,1.0
4676433,2021-07-28,688981.SH,14.893630,78.518421,3.492594,3.670410,3.639087,3.421894,3.279517,-3.122310,3.135021,2.137252,0.807052,0.464591,0.353287,0.316300,0.296494,1.0
4676434,2021-07-29,688981.SH,12.349971,67.086121,2.818110,3.780901,3.816462,3.554204,3.328144,1.987636,2.044304,2.820667,1.525405,0.413642,0.410223,0.347303,0.305242,1.0


## 5. 分开训练集和测试集

In [22]:
df_.set_index(['dt'], inplace = True)
df_.index = pd.to_datetime(df_.index)
df_

,kdcode,volume,turnover,turn,vwap,vwap_mean3,vwap_mean5,vwap_mean10,pct_chg,chg_mean3,chg_mean5,chg_mean10,adj_open,open_mean3,open_mean5,open_mean10,label
dt,,,,,,,,,,,,,,,,,
2016-01-04,000001.SZ,4.523495,4.509965,-0.370305,0.064544,0.064377,0.064083,0.063036,-2.155881,-3.487342,-4.358350,-6.033728,18.997222,18.994527,18.993194,18.989172,0.0
2016-01-05,000001.SZ,5.411601,5.229687,-0.333054,0.040246,0.052216,0.051918,0.050869,0.238673,-1.550633,-1.937923,-2.682873,17.793307,18.392643,18.391340,18.387424,0.0
2016-01-06,000001.SZ,4.098086,3.990511,-0.388159,0.046357,0.050202,0.049903,0.048853,0.446044,-0.793249,-0.991373,-1.372463,18.040687,18.274465,18.273168,18.269272,0.0
2016-01-07,000001.SZ,1.063213,0.989032,-0.515473,0.022451,0.036157,0.042911,0.041859,-2.003287,-0.710970,-1.755996,-2.431012,18.024195,17.950162,18.209960,18.206075,0.0
2016-01-08,000001.SZ,6.161607,5.803036,-0.301578,0.020267,0.029490,0.038279,0.037226,0.641678,-0.493671,-1.145352,-1.585633,17.694355,17.917182,18.106078,18.102212,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-26,688981.SH,8.201142,39.952081,1.883530,3.249302,3.147030,3.108206,3.194988,4.010486,1.877637,1.651585,0.186159,0.242811,0.255329,0.246581,0.283132,1.0
2021-07-27,688981.SH,26.924907,150.272891,6.675454,3.987435,3.436396,3.303080,3.251229,4.926051,4.373418,3.619566,1.481603,0.353701,0.287027,0.272151,0.284710,1.0
2021-07-28,688981.SH,14.893630,78.518421,3.492594,3.670410,3.639087,3.421894,3.279517,-3.122310,3.135021,2.137252,0.807052,0.464591,0.353287,0.316300,0.296494,1.0


In [23]:
dateSepVal = pd.to_datetime("2020-01-01")
train_df = df_[df_.index < dateSepVal]
test_df = df_[df_.index > dateSepVal]

## 6. 保存到文件

In [24]:
train_df.to_csv('trainData4.csv')

In [25]:
test_df.to_csv('testData4.csv')